In [1]:
import torch
import os
from dataloader_helper import *
from models import *
import numpy as np
import random
import torch.optim as optim
from sklearn.metrics import roc_auc_score
import joblib
import time


torch.backends.cudnn.deterministic = True
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

start = '/data2/meerak/models/'

X_test, y_test = np.array(joblib.load('/data2/meerak/MIMIC_CXR_FTS/UPDATED_test_fts.joblib')), np.array(joblib.load('/data2/meerak/MIMIC_CXR_FTS/UPDATED_test_ys.joblib'))
test_dg = CXRDataset('test', {'features':torch.tensor(X_test), 'labels':torch.tensor(y_test)})
test_loader = DataLoader(test_dg,batch_size = 1,shuffle = False)

def get_epoch(name):
    title = 'best_%s.txt'%(name)
    with open(title) as f:
        count = 0
        vals = []
        tests = []
        for line in f:
            if count > 1:
                vals.append(float(line.split(', ')[4]))
                tests.append(float(line.split(', ')[5]))
            if count == 1:
                curr_params = line.split('\n')[0]
            count += 1
    return np.argmax(vals)


/home/meerak/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/meerak/cxr_mimic/best_pretraining_Cardiomegaly/irpe.py:14: UserWarning: [WARNING] The module `rpe_ops` is not built. For better training performance, please build `rpe_ops`.
  warnings.warn(RED_STR.format("[WARNING] The module `rpe_ops` is not built. \


### TRANSMIL

In [2]:
cD = 128
model = TransMIL(cD).to(device)
epoch = get_epoch('transmil_nope')
modelname = start + 'best_transmil_mimiccxr_densenet_epoch%d'%epoch
model.load_state_dict(torch.load(modelname))

bag_preds = []
bag_ys = []
start_epoch = time.time()
for batch_idx, (curridx, data, target) in enumerate(test_loader):
    data = data.to(device).squeeze(0)
    target = target.to(device)

    bag_prediction = model(data)
    
    bag_preds.extend(F.softmax(bag_prediction, dim = 1)[:, 1].detach().cpu().numpy())
    bag_ys.extend(target.detach().cpu().numpy())
    del data
    del bag_prediction
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print('Test AUROC:', roc_auc_score(bag_ys, bag_preds))
print(end_epoch - start_epoch)

curr_rocs = []
for _ in range(1000):
    curr_idxs = np.random.choice(len(bag_preds), len(bag_preds))
    curr_bp = bag_preds[curr_idxs]
    curr_y = bag_ys[curr_idxs]
    curr_rocs.append(roc_auc_score(curr_y, curr_bp))

curr_rocs.sort()
print('%0.3f (%0.3f, %0.3f)'%(curr_rocs[500], curr_rocs[25], curr_rocs[975]))
joblib.dump(curr_rocs, 'transmil.joblib')

Test AUROC: 0.7983857745524279
116.00493359565735
0.798 (0.791, 0.805)


['transmil.joblib']

In [ ]:
cD = 128
model = TransMIL_LinearPE(cD).to(device)
epoch = get_epoch('transmil_LINEARPE')
modelname = start + 'best_transmil_LINEARPE_mimiccxr_densenet_epoch%d'%epoch
model.load_state_dict(torch.load(modelname))

bag_preds = []
bag_ys = []
start_epoch = time.time()
for batch_idx, (curridx, data, target) in enumerate(test_loader):
    data = data.to(device).squeeze(0)
    target = target.to(device)

    bag_prediction = model(data)
    
    bag_preds.extend(F.softmax(bag_prediction, dim = 1)[:, 1].detach().cpu().numpy())
    bag_ys.extend(target.detach().cpu().numpy())
    del data
    del bag_prediction
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print('Test AUROC:', roc_auc_score(bag_ys, bag_preds))
print(end_epoch - start_epoch)

curr_rocs = []
for _ in range(1000):
    curr_idxs = np.random.choice(len(bag_preds), len(bag_preds))
    curr_bp = bag_preds[curr_idxs]
    curr_y = bag_ys[curr_idxs]
    curr_rocs.append(roc_auc_score(curr_y, curr_bp))

curr_rocs.sort()
print('%0.3f (%0.3f, %0.3f)'%(curr_rocs[500], curr_rocs[25], curr_rocs[975]))
joblib.dump(curr_rocs, 'transmil_linearpe.joblib')